<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ACF and PACF Functions in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>ACF</b></p>
<p style = 'font-size:16px;font-family:Arial'> The ACF() function calculates the autocorrelation or autocovariance of a time series. The autocorrelation and autocovariance show how the time series correlates or
    covaries with itself when delayed by a lag in time or space. When the ACF() function is computed, a coefficient corresponding to a particular lag is affected by all the previous lags. For example, the coefficient for lag 4 includes effects of activity at lags 3, 2, and 1.</p>
 
<p style = 'font-size:18px;font-family:Arial'><b>PACF</b></p>
<p style = 'font-size:16px;font-family:Arial'> The PACF() function provides insight as to whether the function   being modeled is stationary or not. The partial auto correlations are used to measure the degree of correlation between series sample points. The algorithm removes the effects of the previous lag. For example, the coefficient for lag 4 focuses on the effect of activity based only at lag 4, with effects of lags 3, 2, and 1 removed.</p>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    Resample,
    ACF,
    PACF,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_ACF_PACF_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_SalesForecastingUAF_cloud');"        # Takes 1 minute
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_SalesForecastingUAF_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
sales_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Weekly_Sales'))
feature_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Features')).drop(['IsHoliday'], axis=1)
store_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Stores'))

<p style = 'font-size:16px;font-family:Arial'>We will join datasets to create the Analytic Data Set using tdml for basic dataframe manipulations</p>

In [ ]:
# Join store_data with sales_data
sales_data = (
    sales_data.join(store_data, on='Store', how='left', lprefix='t1', rprefix='t2')
    .drop(['t2_Store'], axis=1)
)
sales_data = sales_data.assign(Store=sales_data['t1_Store'])
sales_data = sales_data.drop(['t1_Store'], axis=1)

# Join feature_data with sales_data
sales_data = (
    sales_data.join(feature_data, on=['Store', 'Date'], how='left', lprefix='t1', rprefix='t3')
    .drop(['t3_Store'], axis=1)
    .drop(['t3_Date'], axis=1)
)
sales_data = sales_data.assign(Store=sales_data['t1_Store'])
sales_data = sales_data.assign(Date=sales_data['t1_Date'])
sales_data = sales_data.drop(['t1_Store'], axis=1)
sales_data = sales_data.drop(['t1_Date'], axis=1)

<p style = 'font-size:16px;font-family:Arial'>We will creating time series identifier for partitioning the data. We will join the department and store ID which provides a unique ID for every time series to create a column for partitioning.</p>

In [ ]:
# Assign a new column 'idcols' based on string concatenation
sales_data = sales_data.assign(idcols=sales_data.Dept.str.strip() + '-' + sales_data.Store.str.strip())
sales_data = sales_data.assign(idcols=sales_data.idcols.cast(type_=VARCHAR(10)))

# Check the shape of the DataFrame
sales_data.shape

<p style = 'font-size:16px;font-family:Arial'>The dataset we created contains more than 421k rows and 19 columns. This final dataset will be copied to Vantage database</p>

In [ ]:
copy_to_sql(df = sales_data, table_name = "az_sf_joined", if_exists = "replace")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. ACF</b>
<p style = 'font-size:16px;font-family:Arial'>ACF() function is used to calculate the autocorrelation of a time series with itself by using "max_lags". Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(ACF)

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the time series data into TDSeries, which is required for the input time series.</p>

In [ ]:
from teradataml import TDSeries, Resample

data_series_df = TDSeries(
    data=sales_data,
    id="idcols",
    row_index=("times"),
    row_index_style="TIMECODE",
    payload_field="Weekly_Sales",
    payload_content="REAL"
)
uaf_out1 = Resample(data=data_series_df,
                    interpolate='LINEAR',
                    timecode_start_value="TIMESTAMP '2010-02-05 00:00:00'",
                    timecode_duration="WEEKS(1)")

df=uaf_out1.result
df1=df.select(['idcols','ROW_I', 'Weekly_Sales']).assign(Sales_Date=df.ROW_I)
df1

In [ ]:
data_series_df_acf = TDSeries(data=df1,
                              id="idcols",
                              row_index="Sales_Date",
                              row_index_style="TIMECODE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL",
                              interval="WEEKS(1)") 

In [ ]:
# Normalize the series by removing the unit roots.
from teradataml import ACF
uaf_out = ACF(data=data_series_df,
                  max_lags=12,
                  demean=True,
                  qstat = False,
                  alpha=0.05)
acf_df = uaf_out.result
acf_df

In [ ]:
acf_df_plot = acf_df.loc[acf_df.idcols == '1-10']

In [ ]:
plot = acf_df_plot.plot(x=acf_df_plot.ROW_I, 
        y=(acf_df_plot.OUT_Weekly_Sales, acf_df_plot.CONF_OFF_Weekly_Sales),
        kind='corr', figsize=(600,400),ylabel = " ",
        color="blue",title="Auto Correlation")
plot.show()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. PACF</b>
<p style = 'font-size:16px;font-family:Arial'>We will calculate the partial autocorrelation function coefficients using 'LEVINSON_DURBIN' algorithm, with maximum of 12 lags. Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(PACF)

 
<p style = 'font-size:16px;font-family:Arial'>We will use the same TDSeries created in the step above.</p>

In [ ]:
pacf_out = PACF(data=data_series_df,
                    algorithm='LEVINSON_DURBIN',
                    max_lags=12,
             alpha=0.05)

pacf_df = pacf_out.result
pacf_df

In [ ]:
pacf_df_plot = pacf_df.loc[pacf_df.idcols == '1-10']

In [ ]:
plot = pacf_df_plot.plot(x=pacf_df_plot.ROW_I, 
        y=(pacf_df_plot.OUT_Weekly_Sales, pacf_df_plot.CONF_OFF_Weekly_Sales),
        kind='corr', figsize=(600,400),ylabel = " ",
        color="blue",title="Partial Auto Correlation")
plot.show()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_table(table_name='az_sf_joined')

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_SalesForecastingUAF');" 
#Takes 45 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>ACF function reference: <a href = 'https://docs.teradata.com/search/all?query=ACF&content-lang=en-US'>here</a></li>
    <li>PACF function reference: <a href = 'https://docs.teradata.com/search/all?query=PACF&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>